In [1]:
import pymongo
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sb
import dask.dataframe as ddf

client = pymongo.MongoClient()
db = client.VentDB
breathData = db.breath_collection
RN = db.RN_collection

%matplotlib inline

In [2]:
def bin_samples(x):
    if x <= 0.05:
        return 0
    elif x == np.nan:
        return np.nan
    elif x > 0.05:
        return 1

In [3]:
#breathData.find().count()

In [4]:
patient_df = pd.read_csv('C:\Research_data\Demographic Data v3.csv', engine = 'c',
                             usecols = ['Study ID', 'Age', 'Gender', 'P/F Ratio', 'NMB', 'Start_End_NMB', 'End MV Date/Time', 'Hospital Discharge Date/Time', 'Discharge Location'])
patient_df.set_index(['Study ID'], inplace = True, drop=False)
patient_df.head(15)
patient_df.to_hdf('c:\Research_data\Analysis\Lagged_DS\ds_patient_test.h5', key='patient', format='table', append=False)

In [5]:

patient_df['End MV Date/Time'] = pd.to_datetime(patient_df['End MV Date/Time'])

In [6]:
patient_df.head()

,Study ID,Age,Gender,Hospital Discharge Date/Time,End MV Date/Time,NMB,Start_End_NMB,P/F Ratio,Discharge Location
Study ID,,,,,,,,,
100,100,25,0,4/25/2014 15:37,2014-04-22 14:02:00,0,NaN,225.000000,1
101,101,65,0,5/2/2014 14:40,2014-04-27 10:45:00,0,NaN,75.555556,0
102,102,49,1,5/19/2014 13:17,2014-05-12 11:29:00,0,NaN,101.000000,1
103,103,58,1,6/3/2014 16:02,2014-05-29 15:56:00,0,NaN,195.000000,1
104,104,42,0,7/19/2014 14:10,2014-07-17 16:20:00,1,"7/6/14 17:00, 7/7/14 21:00",142.857143,1


In [7]:
#data = breathData.find({},
#                       {'patient_id': 1, 'date_time': 1, 'breath_num': 1, 'breath_settings.peep': 1,
#                        'breath_settings.fio2': 1, 'breath_settings.set_VT': 1, 'breath_character.elapse_time':1, 
#                        'breath_character.peak_vol':1, 'breath_settings.peak_paw': 1, 'label.' + ds_types: 1, '_id': 1})


#rn = RN.find({'$or': [{'FiO2': {'$exists': 1}}, {'PEEP': {'$exists': 1}}]},
#             {'patientID': 1, 'date_time': 1, 'FiO2': 1, 'PEEP': 1, 'SpO2': 1, 'Set Vt': 1, 'Plat':1,
#              '_id': 0})

In [8]:
#df = pd.io.json.json_normalize(data)
#rn_df = pd.io.json.json_normalize(list(rn))

In [9]:
'''
df['date_time'] = pd.to_datetime(df['date_time'], unit='s').dt.tz_localize('UTC').dt.tz_convert('US/Mountain').dt.tz_localize(None)
df.dropna(axis=0, how='all', subset=['breath_settings.fio2', 'breath_settings.peep'], inplace=True)
df.drop_duplicates(subset = 'date_time', keep = 'last', inplace = True)
df.set_index(['patient_id'], inplace = True, drop = False)
df.sort_index(inplace = True)
df['breath_settings.peep'] = df['breath_settings.peep'].astype(np.float64)
df['breath_num'] = df['breath_num'].astype(np.float64)
df['label.' + ds_types] = df['label.' + ds_types].astype(np.float64)

df.rename(columns={'breath_settings.fio2':'fio2', 'breath_settings.peep': 'peep',
                        'breath_settings.set_VT': 'set_vt', 'breath_character.elapse_time': 'elapse_time', 
                        'breath_character.peak_vol': 'peak_vol', 'breath_settings.peak_paw': 'peak_paw'}, inplace=True)

df = df[(df['elapse_time'] < 10000) & \
        (df['peak_vol'] < 1000) & (df['peak_vol'] > 100)]
'''

"\ndf['date_time'] = pd.to_datetime(df['date_time'], unit='s').dt.tz_localize('UTC').dt.tz_convert('US/Mountain').dt.tz_localize(None)\ndf.dropna(axis=0, how='all', subset=['breath_settings.fio2', 'breath_settings.peep'], inplace=True)\ndf.drop_duplicates(subset = 'date_time', keep = 'last', inplace = True)\ndf.set_index(['patient_id'], inplace = True, drop = False)\ndf.sort_index(inplace = True)\ndf['breath_settings.peep'] = df['breath_settings.peep'].astype(np.float64)\ndf['breath_num'] = df['breath_num'].astype(np.float64)\ndf['label.' + ds_types] = df['label.' + ds_types].astype(np.float64)\n\ndf.rename(columns={'breath_settings.fio2':'fio2', 'breath_settings.peep': 'peep',\n                        'breath_settings.set_VT': 'set_vt', 'breath_character.elapse_time': 'elapse_time', \n                        'breath_character.peak_vol': 'peak_vol', 'breath_settings.peak_paw': 'peak_paw'}, inplace=True)\n\ndf = df[(df['elapse_time'] < 10000) &         (df['peak_vol'] < 1000) & (df['pea

In [10]:
#rn_df['date_time'] = pd.to_datetime(rn_df['date_time'], format='%m/%d/%Y %H:%M:%S')
#rn_df.rename(columns={'patientID':'patient_id', 'FiO2':'fio2', 'PEEP':'peep', 'Set Vt': 'set_vt', 'SpO2':'spo2', 'Plat':'plat'}, inplace=True)
#rn_df.dropna(axis=0, how='all', subset=['fio2', 'peep'], inplace=True)
#rn_df.drop_duplicates(subset = 'date_time', keep = 'last', inplace = True)
#rn_df.set_index(['patient_id', 'date_time'], inplace = True, drop=False)

In [11]:
#df.set_index(['patient_id', 'date_time'], inplace=True, drop=False)
#df.sort_index(inplace=True)
#df.set_index(['patient_id'], inplace=True, drop=False)

In [12]:
#df.to_hdf('c:\Research_data\Analysis\Lagged_DS\ds_breath_data.h5', key='breath', format='table', append=False)
#rn_df.reset_index(inplace=True, drop=True)
#rn_df.to_hdf('c:\Research_data\Analysis\Lagged_DS\ds_rn_data.h5', key='rn', format='table', append=False)
#rn_df.set_index(['patient_id', 'date_time'], inplace = True, drop=False)

In [13]:
df = pd.read_hdf('c:\Research_data\Analysis\pandas_breath_dump_5_1_16.h5', 
                 columns=['patient_id', 'date_time', 'breath_num', 'breath_settings.peep','breath_settings.fio2',
                          'breath_settings.set_VT', 'breath_character.elapse_time', 'breath_character.peak_vol', 
                          'breath_settings.peak_paw', 'ds_predict', 'pfl_predict', 'pvt_predict', 'ie_predict',
                          'breath_character.mean_insp_flow', 'breath_character.mean_exp_flow'])
rn_df = pd.read_hdf('c:\Research_data\Analysis\pandas_rn_dump_5_1_16.h5',
                   columns=['patientID', 'date_time', 'FiO2', 'PEEP', 'SpO2', 'Set Vt', 'Plat', 'RASS'])


In [14]:
df.rename(columns={'breath_settings.fio2':'fio2', 'breath_settings.peep': 'peep',
                        'breath_settings.set_VT': 'set_vt', 'breath_character.elapse_time': 'elapse_time', 
                        'breath_character.peak_vol': 'peak_vol', 'breath_settings.peak_paw': 'peak_paw'}, inplace=True)
rn_df.rename(columns={'patientID':'patient_id', 'FiO2':'fio2', 'PEEP':'peep', 'Set Vt': 'set_vt', 'SpO2':'spo2', 
                      'Plat':'plat'}, inplace=True)

In [15]:
df.dtypes

patient_id                                  int64
date_time                          datetime64[ns]
breath_num                                  int64
peep                                      float64
fio2                                      float64
set_vt                                    float64
elapse_time                                 int64
peak_vol                                  float64
peak_paw                                  float64
ds_predict                                float64
pfl_predict                               float64
pvt_predict                               float64
ie_predict                                float64
breath_character.mean_insp_flow           float64
breath_character.mean_exp_flow            float64
dtype: object

In [16]:
rn_df.dtypes

patient_id             int64
date_time     datetime64[ns]
fio2                 float64
peep                 float64
spo2                 float64
set_vt               float64
plat                 float64
RASS                 float64
dtype: object

In [17]:
df.count()

patient_id                         1904166
date_time                          1904166
breath_num                         1904166
peep                               1869720
fio2                               1869402
set_vt                             1497097
elapse_time                        1904166
peak_vol                           1904166
peak_paw                           1869369
ds_predict                         1903815
pfl_predict                        1903067
pvt_predict                        1903067
ie_predict                         1903057
breath_character.mean_insp_flow    1903978
breath_character.mean_exp_flow     1904098
dtype: int64

In [18]:
rn_df.count()

patient_id    13979
date_time     13979
fio2           2710
peep           2582
spo2          11761
set_vt         1739
plat            589
RASS           2288
dtype: int64

In [19]:
df.date_time.min()

Timestamp('2005-07-01 14:55:27')

In [20]:
df.date_time.max()

Timestamp('2015-11-10 06:09:52')

In [21]:
rn_df.date_time.min()

Timestamp('2014-04-11 19:43:00')

In [22]:
rn_df.date_time.max()

Timestamp('2015-11-17 15:51:00')

In [23]:
df.shape

(1904166, 15)

In [24]:
df = df[(df['elapse_time'] < 10000) & (df['peak_vol'] < 1000) & (df['peak_vol'] > 100)]
df.dropna(subset=['ds_predict', 'pfl_predict','pvt_predict', 'ie_predict'], how='all', inplace=True, axis=0)
df = df[df['breath_character.mean_insp_flow'] > df['breath_character.mean_exp_flow']]
df.drop(['breath_character.mean_insp_flow', 'breath_character.mean_exp_flow'], inplace=True, axis=1)

In [25]:
df.shape

(1885313, 13)

In [26]:
df.count()

patient_id     1885313
date_time      1885313
breath_num     1885313
peep           1851639
fio2           1851331
set_vt         1489302
elapse_time    1885313
peak_vol       1885313
peak_paw       1851301
ds_predict     1885043
pfl_predict    1884576
pvt_predict    1884576
ie_predict     1884575
dtype: int64

In [41]:
import numba

lag_list = 6        #needs to be 36/resmaple_freq
resample_freq = '6H'   #hours to resample to

@numba.vectorize(nopython=True)
def link_groups(rowx, rowy):
    if np.isnan(rowx):
        return rowy
    else:
        return rowx

grouped_df = df.groupby('patient_id')
grouped_rn_df = rn_df.dropna(subset=['fio2', 'peep', 'set_vt'], how='all', axis=0).groupby('patient_id')

resampled_df = pd.DataFrame()

for name, group in grouped_df:
    print('patient_id ', name)
    patient_info = patient_df.ix[name]
   
    try:
        rn_group = grouped_rn_df.get_group(name)
        group = pd.merge(group, rn_group, left_on='date_time', right_on='date_time', how='outer', indicator=True)

        for items in ['fio2', 'peep', 'set_vt', 'patient_id']:
            group[items] = link_groups(group[items+'_x'].values, group[items+'_y'].values)
            group.drop([items+'_x', items+'_y'], inplace=True, axis=1)
        #print(group['_merge'].value_counts())    
    except KeyError as e:
        print(name, 'RN is missing ', e)  

    group.drop_duplicates(subset='date_time', keep='first', inplace=True)
    group.set_index(['date_time'], inplace = True, drop = False, verify_integrity=True)
    group.sort_index(inplace=True)
    group = group[group.index < patient_info['End MV Date/Time']]
    
    group = group.resample(resample_freq).agg({'ds_predict':np.sum, 'pfl_predict':np.sum, 'pvt_predict':np.sum, 'ie_predict':np.sum,
                                                   'breath_num':np.count_nonzero, 'patient_id': np.max, 
                                                   'elapse_time':np.mean, 'fio2': [np.mean, np.max, np.min, np.std], 
                                                   'peep': [np.mean, np.max, np.min, np.std], 
                                                   'peak_paw':[np.mean, np.max, np.min, np.std], 
                                                   'set_vt':[np.mean, np.max, np.min, np.std], 
                                                   'peak_vol':[np.mean, np.max, np.min, np.std],
                                                   'RASS':[np.mean, np.max, np.min, np.std]}, axis=1)
    group['NMB'] = 0

    start = group.index.min()
    end = group.index.max()

    periods = (end - start) / pd.to_timedelta(resample_freq)
    periods = periods + lag_list + 1

    group = group.reindex(pd.date_range(start, periods = periods, freq=resample_freq), copy=True)    
    group.reset_index(drop=False, inplace=True)
    group.rename(columns={'index':'date_time'}, inplace=True)
    group.set_index(['date_time'], drop=False, inplace=True, verify_integrity=True)

    #linear extrapolation
    #for items in ['fio2', 'peep', 'set_vt', 'peak_paw']:
    #    group[items].interpolate(method='linear', limit=lag_list, limit_direction='forward', inplace=True)
    #group.patient_id = name
    
    #mortality extrapolation
    #try:   
    #    if patient_info['Discharge Location'] == 0:
    #        print('dead')
    #        group.loc[group.index > end, ('fio2', 'mean')] = 100
    #        group.loc[group.index > end, ('peep', 'mean')] = 18
    #    else:
    #        if group.index.max() > patient_info['End MV Date/Time']:
    #            group.loc[group.index > end, ('fio2', 'mean')] = 25
    #            group.loc[group.index > end, ('peep', 'mean')] = 2
    #except Exception as e:
    #    print('error ', e)
            
    try:
        patient_info = patient_df.ix[name]
        if patient_info['NMB'] == 'Yes':
            start_stop = patient_info['Start_End_NMB'].strip('[]').split(';')

            for items in start_stop:
                start, stop = items.strip('()').split(',')
                start = pd.to_datetime(start)
                stop = pd.to_datetime(stop)

                if start < stop:
                    group.loc[(group.index >= start) & (group.index <= stop), 'NMB'] = 1
                else:
                    rn_df.loc[(group.index >= stop) & (group.index <= start), 'NMB'] = 1
        group['NMB_sum'] = group['NMB'].rolling(window=lag_list, center=False).sum() 
    except KeyError:
        print(name, ' is missing from patient data')  
    
    for ds_types in ['ds', 'pfl', 'pvt', 'ie']:
        try:
            group[ds_types+'_freq'] = group[ds_types +'_predict']['sum'] / group['breath_num']['count_nonzero']
        except ZeroDivisionError:
            print(group[ds_types+'_predict', 'breath_num'])

    for lags in [4,6,8]:
        for items in ['ds', 'fio2', 'peep', 'peak_paw', 'set_vt']:
            if items == 'ds':
                for ds_types in ['ds', 'pfl', 'pvt', 'ie']:
                    group[ds_types + '_lag_' + str(lags)] = np.nan
                    group[ds_types + '_lag_' + str(lags)] = group[ds_types+'_freq'].shift(lags)
                    group[ds_types + '_lag_' + str(lags)] = group[ds_types+ '_lag_' + str(lags)].astype(np.float64)
            else:
                for sub_items in ['mean', 'amax', 'amin', 'std']:
                    group[items + '_lag_' + str(lags)+'_'+sub_items] = group[items][sub_items].shift(lags)                
                    group[items + '_diff_lag_' + str(lags)+'_'+sub_items] = group[items][sub_items] - group[items + '_lag_' + str(lags)+'_'+sub_items]

        try:
            group['NMB_lag'+str(lags)] = 0
            group['NMB_lag'+str(lags)] = group['NMB_sum'].shift(lags)
        except KeyError:
            pass

    resampled_df = pd.concat([resampled_df, group])

patient_id  100
patient_id  101
patient_id  102
patient_id  103
patient_id  104
patient_id  105
patient_id  106
patient_id  107
patient_id  108
patient_id  109
patient_id  110
patient_id  112
patient_id  113
patient_id  114
patient_id  115
patient_id  116
patient_id  117
patient_id  118
patient_id  119
patient_id  120
patient_id  121
patient_id  122
patient_id  124
patient_id  125
patient_id  126
patient_id  127
patient_id  128
patient_id  129
patient_id  130
patient_id  131
patient_id  132


In [42]:
resampled_df.describe()

peak_paw                                       ds_predict  \
             mean        amax        amin         std          sum   
count  309.000000  309.000000  309.000000  308.000000   315.000000   
mean    23.287200   36.618123   12.653722    2.290043   161.961905   
std      6.477138    8.388098    8.766441    1.222191   262.085363   
min     10.666667   12.000000    0.000000    0.057307     0.000000   
25%     18.192574   32.000000    7.000000    1.331854    17.500000   
50%     23.222892   37.000000   11.000000    2.121312    56.000000   
75%     28.271747   42.000000   19.000000    3.073619   164.000000   
max     40.143186   62.000000   33.000000    7.966672  1693.000000   

       patient_id        fio2                                       ...     \
             amax        mean        amax        amin         std   ...      
count  651.000000  648.000000  648.000000  648.000000  609.000000   ...      
mean   117.193548   51.545515   55.333333   49.337963    1.741225   ...      
std      9.219846   17.009372   20.366958   15.721543    4.022133   ...      
min    100.000000   30.000000   30.000000    1.000000    0.000000   ...      
25%    110.000000   40.000000   40.000000   40.000000    0.000000   ...      
50%    119.000000   45.206592   50.000000   40.000000    0.000000   ...      
75%    125.000000   60.000000   70.000000   60.000000    0.752495   ...      
max    132.000000  100.000000  100.000000  100.000000   29.754844   ...      

      peak_paw_diff_lag_8_std set_vt_lag_8_mean set_vt_diff_lag_8_mean  \
                                                                         
count              118.000000        488.000000             286.000000   
mean                -0.231349        430.063525              -5.038983   
std                  1.593109         79.868632              41.697564   
min                 -4.496810        270.000000            -165.000000   
25%                 -1.337656        400.000000               0.000000   
50%                 -0.194296        450.000000               0.000000   
75%                  0.865435        480.000000               0.000000   
max                  3.758764        630.000000             160.000000   

      set_vt_lag_8_amax set_vt_diff_lag_8_amax set_vt_lag_8_amin  \
                                                                   
count        488.000000             286.000000        488.000000   
mean         433.514344              -6.993007        427.081967   
std           79.841963              42.460053         81.693833   
min          270.000000            -200.000000        140.000000   
25%          400.000000               0.000000        380.000000   
50%          470.000000               0.000000        450.000000   
75%          490.000000               0.000000        480.000000   
max          630.000000             160.000000        630.000000   

      set_vt_diff_lag_8_amin set_vt_lag_8_std set_vt_diff_lag_8_std NMB_lag8  
                                                                              
count             286.000000       441.000000            231.000000  13647.0  
mean               -5.314685         2.208919              0.293764      0.0  
std                53.742521        14.388847             21.033510      0.0  
min              -330.000000         0.000000           -134.164079      0.0  
25%                 0.000000         0.000000              0.000000      0.0  
50%                 0.000000         0.000000              0.000000      0.0  
75%                 0.000000         0.000000              0.000000      0.0  
max               300.000000       233.345238            233.345238      0.0  

[8 rows x 148 columns]

In [43]:
for lag_list in [4,6,8]:
    for sub_items in ['mean', 'amax', 'amin', 'std']:
        resampled_df['peep_diff_lag_'+str(lag_list)+'_'+sub_items] = resampled_df['peep'][sub_items] - resampled_df['peep_lag_'+str(lag_list)+'_'+sub_items]
        resampled_df['fio2_diff_lag_'+str(lag_list)+'_'+sub_items] = resampled_df['fio2'][sub_items] - resampled_df['fio2_lag_'+str(lag_list)+'_'+sub_items]
        resampled_df['set_vt_diff_lag_'+str(lag_list)+'_'+sub_items] = resampled_df['set_vt'][sub_items] - resampled_df['set_vt_lag_'+str(lag_list)+'_'+sub_items]
        resampled_df['peak_paw_diff_lag_'+str(lag_list)+'_'+sub_items] = resampled_df['peak_paw'][sub_items] - resampled_df['peak_paw_lag_'+str(lag_list)+'_'+sub_items]

In [44]:
for times in ['4','6','8']:
    for ds_types in ['ds', 'pfl', 'pvt', 'ie']:
        resampled_df[ds_types+'_lag_' + times + '_bin'] = resampled_df[ds_types+'_lag_' + times].apply(bin_samples)

resampled_df.to_csv('c:\\Research_data\\Analysis\\Lagged_DS\\5-1-16_data\\No_Extrap\\lagged_analysis_'+resample_freq+'.csv')
resampled_df.to_hdf('c:\\Research_data\\Analysis\\Lagged_DS\\5-1-16_data\\No_Extrap\\lagged_analysis_'+resample_freq+'.h5', 'table', append=False)